In [ ]:
import requests
import json

# API endpoint and params
url = "https://uqn.gov.sa/News/GetNewsItems"
params = {
    "WebPageId": "12",
    "NewsCount": "1000",
    "HasMediaOnly": "false",
    "LoadMediaThumbnail": "false",
    "IsNext": "false"
}

# Send request
response = requests.get(url, params=params)
\
if response.status_code == 200:
    try:
        # First parse as JSON (will give a string)
        raw_text = response.json()  # This is still a string
        print("🔍 Raw type:", type(raw_text))

        # Decode the actual JSON string
        items = json.loads(raw_text)

        # Extract NewsId from each item
        ids = [item["NewsId"] for item in items]
        urls = [f"https://uqn.gov.sa/details?p={id_}" for id_ in ids]

        # Output
        for url in urls:
            print(url)

        with open("uqn_cat12_links.txt", "w", encoding="utf-8") as f:
            f.write("\n".join(urls))

        print(f"\n✅ Saved {len(urls)} links from Category 12.")

    except Exception as e:
        print("❌ Error while parsing nested JSON:", e)
else:
    print(f"❌ HTTP {response.status_code}")


🔍 Raw type: <class 'str'>
https://uqn.gov.sa/details?p=27216
https://uqn.gov.sa/details?p=27215
https://uqn.gov.sa/details?p=27214
https://uqn.gov.sa/details?p=27213
https://uqn.gov.sa/details?p=27209
https://uqn.gov.sa/details?p=27179
https://uqn.gov.sa/details?p=27150
https://uqn.gov.sa/details?p=27084
https://uqn.gov.sa/details?p=27039
https://uqn.gov.sa/details?p=26991
https://uqn.gov.sa/details?p=26990
https://uqn.gov.sa/details?p=26969
https://uqn.gov.sa/details?p=26903
https://uqn.gov.sa/details?p=26869
https://uqn.gov.sa/details?p=26867
https://uqn.gov.sa/details?p=26866
https://uqn.gov.sa/details?p=26831
https://uqn.gov.sa/details?p=26798
https://uqn.gov.sa/details?p=26780
https://uqn.gov.sa/details?p=26706
https://uqn.gov.sa/details?p=26675
https://uqn.gov.sa/details?p=26673
https://uqn.gov.sa/details?p=26633
https://uqn.gov.sa/details?p=26607
https://uqn.gov.sa/details?p=26606
https://uqn.gov.sa/details?p=26605
https://uqn.gov.sa/details?p=26574
https://uqn.gov.sa/details?p=

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.oxml import OxmlElement

# Ensure output folder exists
output_dir = "cat 11"
os.makedirs(output_dir, exist_ok=True)

# Load URLs from file
with open("uqn_cat11_links.txt", "r", encoding="utf-8") as f:
    urls = [line.strip() for line in f if line.strip()]

# Utility: clean and truncate filename
def clean_filename(text, max_length=100):
    clean = "".join(c for c in text if c.isalnum() or c in " _-").strip()
    return clean[:max_length]

# Utility: apply RTL to paragraph
def set_rtl(paragraph):
    p = paragraph._element
    pPr = p.get_or_add_pPr()
    bidi = OxmlElement('w:bidi')
    bidi.set(qn('w:val'), '1')
    pPr.append(bidi)

# Loop over each URL
for url in urls:
    try:
        res = requests.get(url)
        if res.status_code != 200:
            print(f"❌ Failed to fetch {url}")
            continue

        soup = BeautifulSoup(res.content, "html.parser")

        # Extract elements
        title_elem = soup.find("h3", class_="mb-2 p-3 newsHeader")
        date_elem = soup.find("time", class_="td-module-date")
        body_elem = soup.select_one("body > div.container.align-self-start.p-0 > main > div.content.mt-4.mb-4.border-bottom.pb-4")

        # Skip if body or title is missing or empty
        if not title_elem or not body_elem:
            print(f"⚠️ Skipping due to missing content at {url}")
            continue

        title = title_elem.get_text(strip=True)
        if not title:
            print(f"⚠️ Skipping due to empty title at {url}")
            continue

        date = date_elem.get_text(strip=True) if date_elem else ""
        body_text = body_elem.get_text(separator="\n", strip=True)
        if not body_text:
            print(f"⚠️ Skipping due to empty body at {url}")
            continue

        # Create DOCX
        doc = Document()
        doc.styles['Normal'].font.name = 'Arial'
        doc.styles['Normal'].font.size = Pt(12)

        p_title = doc.add_paragraph(title)
        set_rtl(p_title)

        if date:
            p_date = doc.add_paragraph(date)
            set_rtl(p_date)

        p_body = doc.add_paragraph(body_text)
        set_rtl(p_body)

        # Save file with safe fallback and auto-increment suffix if exists
        safe_title = clean_filename(title)
        if not safe_title:
            print(f"⚠️ Skipping due to invalid filename from title at {url}")
            continue

        base_filename = safe_title
        filename = f"{base_filename}.docx"
        file_path = os.path.join(output_dir, filename)
        counter = 2

        while os.path.exists(file_path):
            suffix = f"-{counter}"
            trimmed_title = base_filename[:100 - len(suffix) - 5]
            filename = f"{trimmed_title}{suffix}.docx"
            file_path = os.path.join(output_dir, filename)
            counter += 1

        try:
            doc.save(file_path)
            print(f"✅ Saved: {filename}")
        except OSError:
            fallback_name = f"uqn_{url.split('=')[-1]}.docx"
            fallback_path = os.path.join(output_dir, fallback_name)
            doc.save(fallback_path)
            print(f"⚠️ Title too long or bad name, saved as fallback: {fallback_name}")

    except Exception as e:
        print(f"❌ Error processing {url}: {e}")


✅ Saved: إعلان التسجيل العيني لمناطق عقارية في منطقة مكة المكرمة
✅ Saved: إعلان التسجيل العيني لمناطق عقارية في منطقة مكة المكرمة
✅ Saved: إعلان التسجيل العيني في مناطق عقارية بالمنطقة الشرقية
✅ Saved: إعلان التسجيل العيني في مناطق عقارية بالمنطقة الشرقية
✅ Saved: اعتماد تعديل وثيقة دليل تقديم الخدمة الكهربائية
✅ Saved: اعتماد تعديل وثيقة دليل تقديم الخدمة الكهربائية
✅ Saved: نزع ملكية المرحلة الثانية للمباني في حدود صك الأرض المجاورة لمقر رئاسة أمن الدولة بحي الحمراء بمحافظة جدة
✅ Saved: نزع ملكية المرحلة الثانية للمباني في حدود صك الأرض المجاورة لمقر رئاسة أمن الدولة بحي الحمراء بمحافظة جدة
✅ Saved: تطبيق المرحلة الحادية والعشرين لربط أنظمة الفوترة الإلكترونية
✅ Saved: الموافقة على إعلان التسجيل العيني في المناطق العقارية بالمنطقة الشرقية
✅ Saved: تعديل المدة الواردة في الفقرة (د/1) من البند (رابعاً) من القرار الوزاري رقم (1007) بشأن تطبيق أحكام اللائحة التنفيذية لجباية الزكاة
✅ Saved: تطبيق المرحلة العشرين لربط أنظمة الفوترة الإلكترونية
✅ Saved: سياسة برنامج المشغّل الاقتصادي المعتم

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os

# Change this to your target folder
folder_path = '/content/drive/MyDrive/Temp/cat 7'

# Count all files (not directories) in the folder and its subfolders
file_count = sum(len(files) for _, _, files in os.walk(folder_path))

print(f"Total number of files in '{folder_path}': {file_count}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total number of files in '/content/drive/MyDrive/Temp/cat 7': 12
